In [1]:
import pandas as pd
import parquet
import alphalens as al
from tqdm import tqdm


In [2]:
df=pd.read_parquet('stock_data_w_alphas.parquet')

In [3]:
df=df.dropna()

In [4]:
df = df.drop_duplicates(['date', 'TICKER'], keep='first')

In [6]:
df

,date,TICKER,S_DQ_CLOSE,RET,alpha001,alpha002,alpha003,alpha004,alpha005,alpha006,...,alpha086,alpha088,alpha092,alpha094,alpha095,alpha096,alpha098,alpha099,alpha101,1D_forward_return
124471,2001-01-26,SUNW,31.18750,0.000000,0.433086,0.386525,-0.237328,-5.0,-0.444184,-0.304482,...,0,0.514256,1.0,-0.880423,True,-7.0,0.231610,0,0.227108,-0.070764
124913,2001-01-29,SUNW,33.56250,0.076152,0.856081,-0.503691,-0.304077,-5.0,-0.444717,-0.358804,...,0,0.501056,1.0,-0.807621,True,-7.0,0.050543,0,0.744864,0.046784
125879,2001-01-30,SUNW,32.06250,-0.044693,0.618694,-0.343073,-0.066061,-7.0,-0.447516,-0.218272,...,0,0.524287,1.0,-0.650963,True,-8.0,-0.189888,0,-0.481196,0.049080
126189,2001-01-31,SUNW,30.56250,-0.046784,0.433086,-0.361457,-0.063250,-4.0,-0.446583,-0.238734,...,0,0.523759,1.0,-0.277249,True,-7.0,-0.411044,-1,-0.682660,0.542587
135113,2001-02-27,SUNW,19.81250,-0.045181,0.252788,-0.433718,0.673484,-4.0,-0.433786,0.623345,...,0,0.366420,2.0,-0.454762,True,-8.0,-0.178377,-1,-0.788809,-0.003145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133113,2016-12-22,NTES,217.10001,-0.026981,0.595618,-0.041233,0.266548,-6.0,-0.386488,0.267122,...,0,0.610487,1.0,-0.304183,True,-6.0,0.259210,0,-0.766820,-0.029374
2133490,2016-12-23,NTES,223.67000,0.030263,0.848606,0.304483,-0.068472,-5.0,-0.360475,-0.051078,...,0,0.734082,2.0,-0.212731,True,-7.0,0.091201,0,0.858807,-0.001429
2134129,2016-12-27,NTES,223.99001,0.001431,0.848606,0.142212,0.395631,-9.0,-0.397637,0.454995,...,0,0.434457,3.0,-0.384117,True,-8.0,-0.196392,0,-0.096312,0.000313
2134924,2016-12-28,NTES,223.92000,-0.000313,0.595618,0.095192,0.547896,-8.0,-0.405070,0.586525,...,0,0.322097,1.0,-0.608365,True,-8.0,-0.552966,0,-0.495236,0.004126


In [8]:
# Calculate 1-day forward returns. This could be adjusted to any period you're interested in.
df['1D_forward_return'] = df.groupby('TICKER')['S_DQ_CLOSE'].pct_change(-1)
# Filter for columns needed for Alphalens
factor_names = [col for col in df.columns if col.startswith('alpha')]
factors_data = df[['date', 'TICKER'] + factor_names + ['1D_forward_return']]

# Convert 'date' to datetime format
factors_data['date'] = pd.to_datetime(factors_data['date'])

# Set MultiIndex with date and TICKER
factors_data.set_index(['date', 'TICKER'], inplace=True)


/var/folders/bm/bm_nkxfx3w1303dr6gyz9cg00000gn/T/ipykernel_31764/843912946.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factors_data['date'] = pd.to_datetime(factors_data['date'])


In [9]:
df['date'] = pd.to_datetime(df['date'])
# Convert 'df' into a format suitable for Alphalens, ensuring no timezone for simplicity
prices = df.pivot(index='date', columns='TICKER', values='S_DQ_CLOSE')
prices.index = prices.index.tz_localize(None)  # Ensure timezone-naive index

for alpha in factors_data.columns:
    if alpha.startswith("alpha"):
        # Prepare the factor series, ensuring it is timezone-naive
        factor_series = factors_data[alpha].dropna()
        factor_series.index = pd.MultiIndex.from_frame(factor_series.reset_index()[['date', 'TICKER']])
        # Correct approach to remove timezone information
        new_levels = factor_series.index.levels[0].tz_localize(None)
        factor_series.index = factor_series.index.set_levels(new_levels, level=0)

        # Run Alphalens analysis, ensuring both factor and prices are timezone-naive
        factor_data = al.utils.get_clean_factor_and_forward_returns(factor=factor_series,
                                                                     prices=prices,
                                                                     periods=[1])

        # Output the Alphalens tear sheet for this factor
        print(f"Alphalens Analysis for: {alpha}")
        al.tears.create_full_tear_sheet(factor_data, groupby=None, groupby_labels=None)




/Users/jacobbarcelona/.pyenv/versions/3.9.16/lib/python3.9/site-packages/alphalens/utils.py:288: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change(period)


IndexError: invalid index to scalar variable.